In [4]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [5]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [6]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [7]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [8]:
def getAdjustment():
    method = 'hmy_getCurrentUtilityMetrics'
    params = []
    num = get_information(method, params)['result']['Adjustment']
    return float(num)

In [9]:
def getBlockSigners(blockNum):
    method = 'hmy_getBlockSigners'
    params = [blockNum]
    return get_information(method, params)['result']

In [10]:
def getStakedAmount():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    num = get_information(method, params)['result']['total-staking']
    return int(num)

In [11]:
def check(block_stake, block_reward):
    # reward increasing
    sorted_reward = sorted(block_reward.items(), key=lambda kv: kv[1], reverse = False)
    print(sorted_reward)
    # stake should remain same or decrease
    sorted_stake = sorted(block_stake.items(), key=lambda kv: kv[1], reverse = False)
    print(sorted_stake)
    first_block, first_stake = sorted_stake[0]
    for block, stake in sorted_stake:
        if stake > first_stake:
            return False
    return True

In [14]:
def R8_test():
    print("Test-R8: Block reward inversely proportional to staked amount")
    num = int(input("enter the number of blocks you want to test: "))
    
    if num < 2:
        print("need at least two blocks to compare, rerun the test!")
        return
    
    current_block = getBlockNumber()
    next_block = current_block + 1
    
    acc_rewards_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated
            
    current_block = getBlockNumber()
    while current_block < next_block:
        current_block = getBlockNumber()
    
    block_reward = dict()
    block_stake = dict()
    
    iterations = 0
    while iterations < num:
        print("current block", current_block)
        # staked amount
        staked = getStakedAmount()
        block_stake[current_block] = staked
        # get the validator info and compute validator rewards
        acc_rewards_curr = dict()
        delegations_curr = dict()
        validator_infos = getAllValidatorInformation()
        total_reward = 0
        for i in validator_infos:
            if i['currently-in-committee'] == True:
                address = i['validator']['address']
                reward_accumulated = i['lifetime']['reward-accumulated']
                acc_rewards_curr[address] = reward_accumulated
                reward = reward_accumulated
                if address not in acc_rewards_prev:
                    continue
                reward = reward_accumulated - acc_rewards_prev[address]
                total_reward = total_reward + reward
                
        block_reward[current_block] = total_reward
        
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        acc_rewards_prev = acc_rewards_curr
        delegations_prev = delegations_curr
        
        iterations = iterations + 1
    
    print("size", len(block_stake), len(block_reward))
    if check(block_stake, block_reward) != True:
        print("Test-R8: Failed, block reward is not inversely proportional to stake")
    print("done!")

In [ ]:
R8_test()

Test-R8: Block reward inversely proportional to staked amount
enter the number of blocks you want to test: 20
current block 711
current block 712
current block 713
current block 714
current block 715
current block 716
current block 717
current block 718
current block 719
current block 720
current block 721
current block 722
current block 723
current block 724
current block 725
current block 726
current block 727
current block 728
current block 729
current block 730
